In [3]:
from pymongo import MongoClient
import scipy.io
import numpy as np
import gridfs
import os
from tqdm import tqdm
from glob import glob
from PIL import Image
import io
import pandas as pd

연결테스트

In [ ]:
# MongoDB 클라이언트에 연결
client = MongoClient("mongodb://172.17.0.9:27017")

try:
    # 'admin' 데이터베이스에서 ping 명령 실행
    client.admin.command("ping")
    print("MongoDB 서버에 정상적으로 연결되었습니다.")
except Exception as e:
    print("MongoDB 서버에 연결할 수 없습니다:", e)

데이터삽입
1. boundary (.mat) -> array 추출 -> 저장
2. geometry (.png) -> 저장
3. solve (.csv) -> 저장

In [5]:
# MongoDB 연결 설정
client = MongoClient("mongodb://172.17.0.9:27017")
db = client["simulation_data"]

# 컬렉션 지정
collection_boundary = db["boundary_data"]
collection_solve = db["solve_data"]
collection_preprocessed = db["preprocessed_data"]

# Geometry 데이터는 이미지라서 별도 설정
collection_geometry_fs = gridfs.GridFS(db, collection="geometry_data")

# 데이터 지정
BOUNDARY_PATH = "dataset/raw/Boundary/*"
SOLVE_PATH = "dataset/raw/Solve/*"
GEO_PATH = "dataset/raw/Subd/*"
PREPROCESSED_PATH = "dataset/preprocessed/*"
boundary_lst = sorted(glob(BOUNDARY_PATH))
solve_data_lst = sorted(glob(SOLVE_PATH))
geometry_data_lst = sorted(glob(GEO_PATH))
preprocessed_data_lst = sorted(glob(PREPROCESSED_PATH))

Boundary 데이터 삽입

In [ ]:
# Boundary 데이터 지정
for boundary_path in tqdm(boundary_lst):
    idx = boundary_path.split("/")[-1].split("_")[0]
    boundary = np.array(scipy.io.loadmat(boundary_path)['Boundary'])
    boundary = np.where(boundary==5,0,boundary)

    # 배열을 리스트 형식으로 변환하여 MongoDB에 저장
    boundary_document = {
        "index":idx,
        "data": boundary.tolist(),  # 배열 데이터를 리스트로 저장
        "shape": boundary.shape,    # 배열의 shape 저장
    }

    # 기존 문서가 있는지 확인
    if not collection_boundary.find_one({"index": idx}):
        collection_boundary.insert_one(boundary_document)
    else:
        print(f"{idx} 인덱스의 문서가 이미 존재합니다.")

In [ ]:
# 문서 갯수 확인
document_count = collection_boundary.count_documents({})
print(f"총 문서 개수: {document_count}")

# 데이터결과 한개씩 확인
matching_documents = collection_boundary.find()
for doc in matching_documents:
    print(doc['index'])
    break

In [8]:
matching_documents = collection_boundary.find()
for doc in matching_documents:
    break

Geometry 데이터 삽입

In [ ]:
for geometry_path in tqdm(geometry_data_lst):
    idx = geometry_path.split("/")[-1].split("_")[0]
    if not collection_geometry_fs.find_one({"filename": f"{idx}_geometry.png"}):
        with open(geometry_path, "rb") as f:
            collection_geometry_fs.put(f, filename=f"{idx}_geometry.png")
    else:
        print(f"{idx} 인덱스의 문서가 이미 존재합니다.")

In [10]:
# 문서 갯수 확인
collection_geometry_files = db["geometry_data.files"] # geometry_data는 collection_geometry_fs 생성시 지정했음.
collection_geometry_chunks = db["geometry_data.chunks"] # geometry_data는 collection_geometry_fs 생성시 지정했음.
file_count = collection_geometry_files.count_documents({})
print(f"총 파일 개수: {file_count}")

# 데이터결과 한개씩 확인
file_data = collection_geometry_fs.find_one({"filename": "2100_geometry.png"})
if file_data:
    image = Image.open(io.BytesIO(file_data.read()))
    image.save("retrieved_geometry.png")
    print("이미지 파일을 MongoDB에서 성공적으로 읽어왔습니다.")
else:
    print("해당 파일을 찾을 수 없습니다.")

총 파일 개수: 550
이미지 파일을 MongoDB에서 성공적으로 읽어왔습니다.


In [12]:
collection_geometry_files.find_one()

{'_id': ObjectId('673316538fcc2a7a6951ba75'),
 'filename': '1651_geometry.png',
 'chunkSize': 261120,
 'length': 2117,
 'uploadDate': datetime.datetime(2024, 11, 12, 8, 48, 19, 522000)}

In [20]:
type(collection_geometry_files.find_one()['filename'])

str

solve 데이터 삽입

In [ ]:
# solve 데이터 지정
for solve_path in tqdm(solve_data_lst):
    idx = solve_path.split("/")[-1].split("_")[0]
    df = pd.read_csv(solve_path)

    solve_document = {
        "index":idx,
        "data": df.to_dict('records'),
        "shape": df.shape,
    }

    # 기존 문서가 있는지 확인
    if not collection_solve.find_one({"index": idx}):
        collection_solve.insert_one(solve_document)
    else:
        print(f"{idx} 인덱스의 문서가 이미 존재합니다.")

In [ ]:
# 문서 갯수 확인
document_count = collection_solve.count_documents({})
print(f"총 문서 개수: {document_count}")

# 데이터결과 한개씩 확인
matching_documents = collection_solve.find()
for doc in matching_documents:
    print(doc['index'])
    break

preprocessed_data 삽입

In [ ]:
# solve 데이터 지정
for preprocessed_path in tqdm(preprocessed_data_lst):
    idx = preprocessed_path.split("/")[-1].split("_")[0]
    df = pd.read_csv(preprocessed_path)

    preprocessed_document = {
        "index":idx,
        "data": df.to_dict('records'),
        "shape": df.shape,
    }

    # 기존 문서가 있는지 확인
    if not collection_preprocessed.find_one({"index": idx}):
        collection_preprocessed.insert_one(preprocessed_document)
    else:
        print(f"{idx} 인덱스의 문서가 이미 존재합니다.")

In [ ]:
# 문서 갯수 확인
document_count = collection_preprocessed.count_documents({})
print(f"총 문서 개수: {document_count}")

# 데이터결과 한개씩 확인
matching_documents = collection_preprocessed.find()
for doc in matching_documents:
    print(doc['index'])
    break

In [ ]:
collection_preprocessed.find_one({"index":"1651"}).keys()

삭제

In [37]:
# boundary 데이터 삭제
collection_boundary.delete_many({})

# solve 데이터 삭제
collection_solve.delete_many({})

# geometry 데이터 삭제
file_ids = [file["_id"] for file in db["geometry_data.files"].find()]
for file_id in file_ids:
    collection_geometry_fs.delete(file_id)